In [2]:
import deepchem as dc
import pandas as pd
import numpy as np
from rdkit import Chem

import xgboost as xgb
from sklearn.model_selection import GridSearchCV,KFold

from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (D:\anaconda\envs\PI3K\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
data = pd.read_csv('../../data/refined_gabaa.csv')

# Feature extraction & Data splitting

In [4]:
featurizer = dc.feat.CircularFingerprint(size=1024,radius=4)
features = featurizer.featurize(data['smiles'])
dataset = dc.data.NumpyDataset(features,data['label'])

In [5]:
splitter = dc.splits.RandomSplitter()
train_dataset, test_dataset = splitter.train_test_split(dataset=dataset,frac_train=0.7,seed=100)

# GridSearchCV

In [ ]:
scoring = {
          'F1':make_scorer(f1_score),
          'AUC':make_scorer(roc_auc_score),
           'BA':make_scorer(balanced_accuracy_score),
           'MCC':make_scorer(matthews_corrcoef)
}

param_grid = {'n_estimators':[200],
                  'max_depth':[3,5,7,9,11],
                  'colsample_bytree':[0.5,0.7,0.9],
                 'learning_rate':[0.01,0.05, 0.1,0.15,0.2]
                 }
    
xgb_classifier = xgb.XGBClassifier()

cv = KFold(n_splits=5, shuffle=True, random_state=100)

gs = GridSearchCV(
                   xgb_classifier,
                   param_grid,
                 scoring = scoring,
                  cv = cv,
                 n_jobs = -1,
               refit = 'F1',
               return_train_score = True)

gs_fit = gs.fit(train_dataset.X, train_dataset.y.ravel())

In [6]:
val_F1 = gs.best_score_
val_AUC = gs.cv_results_['mean_test_AUC'][gs.best_index_]
val_BA = gs.cv_results_['mean_test_BA'][gs.best_index_]
val_MCC = gs.cv_results_['mean_test_MCC'][gs.best_index_]

# GridSearchCV_Result

In [7]:
print('Best parameters: ', gs.best_params_)
print('Best score (F1): ', gs.best_score_)
print('AUC: ',gs.cv_results_['mean_test_AUC'][gs.best_index_])
print('BA: ', gs.cv_results_['mean_test_BA'][gs.best_index_])
print('MCC: ',gs.cv_results_['mean_test_MCC'][gs.best_index_])

Best parameters:  {'colsample_bytree': 0.9, 'learning_rate': 0.15, 'max_depth': 7, 'n_estimators': 200}
Best score (F1):  0.8363169326615157
AUC:  0.8579479006560922
BA:  0.8579479006560922
MCC:  0.7185326504643712


In [8]:
svm_model = gs_fit.best_estimator_
svm_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

# Evaluate model

In [9]:
y_test_pred_proba = svm_model.predict_proba(test_dataset.X)
y_test_pred = svm_model.predict(test_dataset.X)

In [10]:
tn, fp, fn, tp = confusion_matrix(test_dataset.y, y_test_pred).ravel()
print('TN:', tn)
print('FP:', fp)
print('FN:', fn)
print('TP:', tp)

TN: 284
FP: 29
FN: 43
TP: 217


In [11]:
test_pred_list = []
for test_score in y_test_pred_proba:
    test_score = test_score[1]
    test_pred_list.append(test_score)

In [12]:
test_pred_array = np.array(test_pred_list)

In [13]:
test_F1 = f1_score(test_dataset.y,np.round(test_pred_array))
test_AUC = roc_auc_score(test_dataset.y,test_pred_list)
test_BA =  balanced_accuracy_score(test_dataset.y,np.round(test_pred_array))
test_MCC = matthews_corrcoef(test_dataset.y,np.round(test_pred_array))

# Finall result

In [14]:
performance_dataset = {
    'F1':[val_F1,test_F1],
    'AUC':[val_AUC,test_AUC],
    'BA':[val_BA,test_BA],
    'MCC':[val_MCC,test_MCC],
}

In [7]:
performance = pd.DataFrame(performance_dataset,index=['val','test'])
performance

NameError: name 'performance_dataset' is not defined